<a href="https://colab.research.google.com/github/atick-faisal/Crowd-Emotion/blob/main/src/tl-generic/Spectrogram_TL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import time
import json
import datetime

import numpy as np
import seaborn as sns
import soundfile as sf
import matplotlib.pyplot as plt

import tensorflow as tf

from scipy.signal import spectrogram
from sklearn.metrics import classification_report, confusion_matrix

from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure

tf.random.set_seed(42)

tf.__version__

'2.4.1'

In [2]:
CONFIG = {
    'timestamp'             : str(datetime.datetime.now()),
    'model'                 : 'Transfer Learning on MobileNet',
    'test_fold'             : 'Fold 1',
    'frame_length'          : 44100,
    'frame_inc'             : 11025,
    'dataset_length'        : 13806,
    'fig_size'              : (2.23, 2.23), # 160x160
    'architecture'          : '',
    'batch_size'            : 1024,
    'epochs'                : 0,
    'learning_rate'         : 0.000005,
    'monitor'               : 'val_loss',
    'patience'              : 3,
    'class_weight'          : { 0: 0.71, 1:  2.32, 2: 0.86 },
    'training_time'         : 0,
    'testing_time'          : 0,
    'cm_atick'              : '',
    'cr_atick'              : '',
    'cm_valentina'          : '',
    'cr_valentina'          : ''
}

In [3]:
BASE_DIR            = os.getcwd()
LOG_FILE             = '/content/drive/MyDrive/Research/Crowd Emotion Logs/tl_yamnet.txt'
FOLDS               = ['Fold 1', 'Fold 2', 'Fold 3', 'Fold 4', 'Fold 5']
EMOTIONS            = ['Approval', 'Disapproval', 'Neutral']

ATICK_DATA_DIR      = '/content/Dataset-Atick/'
VALENTINA_DATA_DIR  = '/content/Dataset-Valentina/'

RANGE               = [20, 20000]

In [4]:
%%time

# # ----------------- Loading my dataset -------------------
# !mkdir /content/Dataset-Atick/
# !gdown --id '1zGfANn9GKi9OUMbMehkfzk_Pvhc-b68N'
# !tar -xf /content/Atick_CE_Dataset.tar.xz -C /content/Dataset-Atick/

# # ----------------- Loading Valentina's dataset -------------------
# !mkdir /content/Dataset-Valentina/
# !gdown --id '11tC2Nmie9v3ljo60oQJ3sN1rVkorV-N1'
# !tar -xf /content/Valentina_CE_Dataset.tar.xz -C /content/Dataset-Valentina/


CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.11 µs


In [5]:
%%time

X = np.zeros((CONFIG['dataset_length'], 160, 160, 3), dtype=np.uint8)

count = 0
for emotion in EMOTIONS:
    emo_path = os.path.join(BASE_DIR, ATICK_DATA_DIR, emotion)

    for fold in FOLDS:
        fold_path = os.path.join(emo_path, fold)
        files = os.listdir(fold_path)
        
        for filename in files:
            wav, sr = sf.read(os.path.join(fold_path, filename))
            n = len(wav)
            idx = 0
            while idx < (n - CONFIG['frame_length']):
                frame = wav[idx:(idx + CONFIG['frame_length'])]
                idx = idx + CONFIG['frame_inc']
                # f, t, specgram = spectrogram(
                #     x                   = frame,
                #     fs                  = 44100,
                #     window              = np.hamming(400),
                #     nperseg             = 400,
                #     noverlap            = 200,
                #     nfft                = 400,
                #     detrend             = None,
                #     return_onesided     = True,
                #     scaling             = 'density',
                #     mode                = 'psd'
                # )

                fig = Figure(figsize=CONFIG['fig_size'])
                canvas = FigureCanvas(fig)
                ax = fig.gca()

                ax.axis('off')
                fig.tight_layout(pad=0)
                ax.margins(0)

                ax.specgram(
                    x               = frame,
                    Fs              = 44100,
                    window          = np.hamming(400),
                    NFFT            = 400,
                    cmap            = 'jet',
                    noverlap        = 200,
                    mode            = 'psd',
                    scale           = 'dB',
                    detrend         = None,
                    scale_by_freq   = True,
                    vmin            = -160,
                    vmax            = -25,
                    interpolation   = 'hamming'      
                )
                ax.set_yscale('symlog')
                ax.set_ylim(RANGE)

                fig.canvas.draw()
                image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
                image_from_plot = image_from_plot.reshape(fig.canvas.get_width_height()[::-1] + (3,))

                X[count, :] = image_from_plot

                count = count + 1

    #         break
    #     break
    # break


ValueError: ignored

In [9]:
fold

'Fold 1'

In [ ]:
fig = Figure(figsize=(2.3, 2.3))
canvas = FigureCanvas(fig)
ax = fig.gca()

ax.axis('off')
fig.tight_layout(pad=0)
ax.margins(0)

psd, _, _, spec_img = ax.specgram(
                    x               = frame,
                    Fs              = 44100,
                    window          = np.hamming(400),
                    NFFT            = 400,
                    cmap            = 'jet',
                    noverlap        = 200,
                    mode            = 'psd',
                    scale           = 'dB',
                    detrend         = None,
                    scale_by_freq   = True,
                    vmin            = -160,
                    vmax            = -25,
                    interpolation   = 'hamming'      
                )
ax.set_yscale('symlog')
ax.set_ylim(RANGE)

fig.canvas.draw()
image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
image_from_plot = image_from_plot.reshape(fig.canvas.get_width_height()[::-1] + (3,))
fig.clear()

In [ ]:
plt.imshow(image_from_plot)

In [ ]:
plt.pcolormesh(t, np.log10(f + 1), 20 * np.log10(specgram), cmap='jet')

In [ ]:
psd, _, _, spec_img = plt.specgram(
                    x               = frame,
                    Fs              = 44100,
                    window          = np.hamming(400),
                    NFFT            = 400,
                    cmap            = 'jet',
                    noverlap        = 200,
                    mode            = 'psd',
                    scale           = 'dB',
                    detrend         = None,
                    scale_by_freq   = True,
                    vmin            = -160,
                    vmax            = -25,
                    interpolation   = 'hamming'      
                )
plt.yscale('symlog')

In [ ]:
CONFIG['frame_length']

In [ ]:
plt.pcolormesh(t, np.log10(f + 1), 20 * np.log10(psd), cmap='jet')

In [ ]:
spec_img